In [36]:
import openpyxl
import ipywidgets as widgets
from IPython.display import display, clear_output
import sounddevice as sd
from scipy.io.wavfile import write
import threading
import time
import os
import numpy as np

# =====================================================================
# BAGIAN 1: KONFIGURASI UMUM & WIDGETS
# =====================================================================

# File Excel
file_path_excel = "data_mahasiswa.xlsx"
header_excel = ["Nama", "Jenis Kelamin", "Prodi", "Asal Kota", "Umur", "Domisili", "Asal Kampus", "Jenis Tes", "Skor Tes", "Persepsi"]

# Perekam suara
fs = 44100  # Sample rate
default_countdown = 30  # countdown persiapan
base_folder_rekaman = "rekaman_i-speak"

# Daftar soal dengan kode singkat
soal_list = [
    ("RASE", "baca""The rapid advancement of technology has transformed the way we communicate with each other."),
    ("RASH", "With a tear in your eye, you will watch as your dress begins to tear."),
    ("RAL", """You don't need to spend all of your hard earned money on bakery bread. Making your own bread at home is easy with the new Double Duty Dough Mixer by Berring. Unlike other bread machines that can be difficult to clean and store, the Double Duty Dough Mixer breaks down into five parts that can go directly into your dishwasher. This stainless steel appliance will mix dough for you in a fraction of the time it takes to knead dough by hand. The automated delay feature at the beginning of the mix cycle gives your ingredients time to reach room temperature, ensuring that your breads will rise as high as bakery bread. We guarantee that the accompanying Berring Best Breads recipe book will be a family favourite."""),
    ("DP", "Describe this picture using complete sentences and clear descriptions. Explain who is in the picture, what is happening, and the overall atmosphere."),
    ("FSDL", "Should people be responsible for what happens because of what they say? Explain with an example."),
    ("FSST", "Talk about the university course you enjoyed the most, describe one course you found difficult, and explain whether you think universities should focus more on practical skills or theoretical knowledge.")
]

# Variabel kontrol
recording = None
stop_flag = False
skip_countdown_flag = False
current_soal_index = 0
folder_path_rekaman = ""

# =====================================================================
# BAGIAN 2: INISIALISASI EXCEL
# =====================================================================
try:
    workbook = openpyxl.load_workbook(file_path_excel)
    sheet = workbook.active
    if sheet.max_row == 0 or [cell.value for cell in sheet[1]] != header_excel:
        sheet.delete_rows(1, sheet.max_row)
        sheet.append(header_excel)
    workbook.save(file_path_excel)
except FileNotFoundError:
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(header_excel)
    workbook.save(file_path_excel)

# =====================================================================
# BAGIAN 3: WIDGETS FORM & PEREKAM
# =====================================================================
# Form data diri
nama = widgets.Text(description="Nama:", layout=widgets.Layout(width='400px'))
jenis_kelamin = widgets.Dropdown(options=["Laki-laki", "Perempuan"], description="Jenis Kelamin:")
program_studi = widgets.Text(description="Prodi:", layout=widgets.Layout(width='400px'))
asal_kota = widgets.Text(description="Asal Kota:", layout=widgets.Layout(width='400px'))
umur = widgets.IntText(description="Umur:")
domisili = widgets.Text(description="Domisili:", layout=widgets.Layout(width='400px'))
asal_kampus = widgets.Text(description="Asal Kampus:", layout=widgets.Layout(width='400px'))
jenis_test = widgets.Dropdown(options=["TOEFL", "IELTS", "Duolingo", "Lainnya", "Belum Pernah"], description="Jenis Tes:")
skor_test = widgets.IntText(description="Skor Tes:", value=0, disabled=True)
persepsi = widgets.Dropdown(options=["Dasar", "Menengah", "Mahir"], description="Persepsi:")
submit_btn = widgets.Button(description="✅ Lanjut ke Sesi Rekaman", button_style="success")
output_form = widgets.Output()

# fungsi enable/disable skor tes
def handle_test_change(change):
    skor_test.disabled = change['new'] == "Belum Pernah"
    if skor_test.disabled:
        skor_test.value = 0

jenis_test.observe(handle_test_change, names='value')
# panggil sekali agar default dropdown langsung sesuai
handle_test_change({'new': jenis_test.value})

# Perekam suara & UI HTML
start_btn = widgets.Button(description="▶️ Mulai Rekam Soal 1", button_style="success")
stop_btn = widgets.Button(description="⏹ Stop Rekam", button_style="danger", disabled=True)
skip_btn = widgets.Button(description="⏭️ Lewati Countdown", button_style="info", disabled=True)
countdown_progress = widgets.IntProgress(value=0, min=0, max=default_countdown,
                                         description='Persiapan:',
                                         bar_style='info',
                                         style={'description_width': 'initial'},
                                         layout=widgets.Layout(width='500px', visibility='hidden'))

soal_ui_html = widgets.HTML(value="<div style='padding: 10px; border: 1px solid #e0e0e0; border-radius: 5px; min-height: 100px;'><i>Menunggu sesi rekaman dimulai...</i></div>")

# =====================================================================
# BAGIAN 4: FUNGSI-FUNGSI
# =====================================================================
def record_audio_for_soal(soal_index):
    global recording, stop_flag, skip_countdown_flag, folder_path_rekaman
    soal_name, soal_text = soal_list[soal_index]
    filename = os.path.join(folder_path_rekaman, f"{nama.value.strip().replace(' ', '_')}_{soal_name}.wav")

    def update_ui(status_html, show_countdown=False):
        gambar_html = ""
        if soal_name == "DP":
            gambar_path = "image/describe.jpg"  # sesuaikan path gambar
            gambar_html = f"<img src='{gambar_path}' style='max-width: 100%; height: auto; border-radius: 5px; margin-bottom: 10px;'>"

        soal_html = f"""
        <div style='border: 1px solid #ccc; padding: 15px; border-radius: 8px; margin-bottom: 10px; background-color: #f9f9f9;'>
            <h4>Soal {soal_index + 1}/{len(soal_list)}: {soal_name}</h4>
            <p style='font-family: serif; font-size: 1.1em; line-height: 1.5;'><i>"{soal_text}"</i></p>
            {gambar_html}
            <hr>
            <p style='text-align: center; font-weight: bold; font-size: 1.2em; margin-top: 15px;'>{status_html}</p>
        </div>
        """
        soal_ui_html.value = soal_html
        countdown_progress.layout.visibility = 'visible' if show_countdown else 'hidden'

    skip_btn.disabled = False

    # Countdown
    for i in range(default_countdown, 0, -1):
        if stop_flag:
            update_ui(f"<span style='color: orange;'>⏹ Rekaman soal '{soal_name}' dibatalkan.</span>")
            skip_btn.disabled = True
            countdown_progress.value = 0
            return
        if skip_countdown_flag:
            update_ui("⏭️ Countdown dilewati!")
            time.sleep(0.5)
            break

        countdown_progress.value = default_countdown - i
        update_ui(f"⏳ Mulai rekam dalam <b>{i} detik...</b>", show_countdown=True)
        time.sleep(1)

    skip_btn.disabled = True
    countdown_progress.value = 0

    if stop_flag: return

    update_ui("<span style='color: red;'>🔴 MEREKAM...</span> Tekan <b>Stop</b> saat selesai.")

    recording_data = []

    def callback(indata, frames, time, status):
        if not stop_flag:
            recording_data.append(indata.copy())

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16', callback=callback):
        while not stop_flag:
            time.sleep(0.1)

    recording = np.concatenate(recording_data, axis=0) if recording_data else np.array([], dtype='int16')

    write(filename, fs, recording)
    update_ui(f"<span style='color: green;'>✅ Rekaman soal '{soal_name}' berhasil disimpan.</span>")
    stop_rekaman(None)

def start_rekaman(b):
    global stop_flag, current_soal_index, skip_countdown_flag
    stop_flag = False
    skip_countdown_flag = False
    start_btn.disabled = True
    stop_btn.disabled = False

    if current_soal_index >= len(soal_list):
        soal_ui_html.value = "<div style='text-align:center; padding: 20px; background-color: #e6ffed; border: 1px solid #b7e4c7; border-radius: 8px;'><h3>🎉 Semua soal sudah direkam. Sesi selesai! ✅</h3></div>"
        start_btn.description = "✅ Selesai"
        stop_btn.disabled = True
        skip_btn.disabled = True
        return

    thread = threading.Thread(target=record_audio_for_soal, args=(current_soal_index,))
    thread.start()

def stop_rekaman(b):
    global stop_flag, current_soal_index
    if not stop_flag:
        stop_flag = True
    stop_btn.disabled = True
    start_btn.disabled = False
    skip_btn.disabled = True

    current_soal_index += 1

    if current_soal_index >= len(soal_list):
        start_btn.description = "✅ Selesai"
        soal_ui_html.value = "<div style='text-align:center; padding: 20px; background-color: #e6ffed; border: 1px solid #b7e4c7; border-radius: 8px;'><h3>🎉 Semua soal sudah direkam. Sesi selesai! ✅</h3></div>"
    else:
        start_btn.description = f"▶️ Lanjut Soal {current_soal_index + 1}"
        # otomatis rekam soal berikutnya
        start_rekaman(None)

def skip_countdown(b):
    global skip_countdown_flag
    skip_countdown_flag = True

def on_submit_form(b):
    global folder_path_rekaman
    if not nama.value.strip():
        with output_form:
            clear_output(wait=True)
            print("⚠️ Nama tidak boleh kosong!")
        return

    sheet.append([
        nama.value, jenis_kelamin.value, program_studi.value, asal_kota.value,
        umur.value, domisili.value, asal_kampus.value, jenis_test.value,
        skor_test.value, persepsi.value
    ])
    workbook.save(file_path_excel)

    # Simpan semua file di satu folder saja
    folder_path_rekaman = base_folder_rekaman
    os.makedirs(folder_path_rekaman, exist_ok=True)

    clear_output(wait=True)
    display(widgets.HTML(f"<h3>👋 Halo, {nama.value}! Data Anda berhasil disimpan.</h3><p>Folder rekaman: <b>'{folder_path_rekaman}'</b><br>Sekarang, silakan mulai sesi rekaman suara di bawah ini.</p>"))
    display(ui_rekaman)

# =====================================================================
# BAGIAN 5: LAYOUT & EVENT HANDLING
# =====================================================================
submit_btn.on_click(on_submit_form)
start_btn.on_click(start_rekaman)
stop_btn.on_click(stop_rekaman)
skip_btn.on_click(skip_countdown)

form_items = widgets.VBox([
    widgets.HTML("<h3>Formulir Data Peserta</h3>"),
    nama, jenis_kelamin, program_studi, asal_kota, umur, domisili, asal_kampus,
    jenis_test, skor_test, persepsi, submit_btn, output_form
])

ui_rekaman = widgets.VBox([
    widgets.HTML("<hr><h3>Sesi Perekaman Suara</h3>"),
    soal_ui_html,
    countdown_progress,
    widgets.HBox([start_btn, stop_btn, skip_btn]),
])

display(form_items)


In [25]:
!pip install streamlit openpyxl sounddevice scipy numpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -------- ------------------------------- 2.1/10.0 MB 11.0 MB/s eta 0:00:01
   ----------------- ---------------------- 4.5/10.0 MB 11.7 MB/s eta 0:00:01
   -------------------------- ------------- 6.6/10.0 MB 11.3 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.0 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 10.5 MB/s  0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 11.3 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------- -------------------------- 2.4/7.0 MB 13.9 MB/s eta 0:00:01
   ------------------------------- -------- 5.5/7.0 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 12.2 MB/s  0:00:00
   ---------------

In [33]:
import openpyxl
import ipywidgets as widgets
from IPython.display import display, clear_output
import sounddevice as sd
from scipy.io.wavfile import write
import threading
import time
import os
import numpy as np

# =====================================================================
# BAGIAN 1: KONFIGURASI UMUM & WIDGETS
# =====================================================================

# File Excel
file_path_excel = "data_mahasiswa.xlsx"
header_excel = ["Nama", "Jenis Kelamin", "Prodi", "Asal Kota", "Umur", "Domisili", "Asal Kampus", "Jenis Tes", "Skor Tes", "Persepsi"]

# Perekam suara
fs = 44100  # Sample rate
default_countdown = 30  # countdown persiapan
base_folder_rekaman = "rekaman_i-speak"

# Daftar soal dengan kode singkat
soal_list = [
    ("RASE", "The rapid advancement of technology has transformed the way we communicate with each other."),
    ("RASH", "With a tear in your eye, you will watch as your dress begins to tear."),
    ("RAL", """You don't need to spend all of your hard earned money on bakery bread. Making your own bread at home is easy with the new Double Duty Dough Mixer by Berring. Unlike other bread machines that can be difficult to clean and store, the Double Duty Dough Mixer breaks down into five parts that can go directly into your dishwasher. This stainless steel appliance will mix dough for you in a fraction of the time it takes to knead dough by hand. The automated delay feature at the beginning of the mix cycle gives your ingredients time to reach room temperature, ensuring that your breads will rise as high as bakery bread. We guarantee that the accompanying Berring Best Breads recipe book will be a family favourite."""),
    ("DP", "Describe this picture using complete sentences and clear descriptions. Explain who is in the picture, what is happening, and the overall atmosphere."),
    ("FSDL", "Should people be responsible for what happens because of what they say? Explain with an example."),
    ("FSST", "Talk about the university course you enjoyed the most, describe one course you found difficult, and explain whether you think universities should focus more on practical skills or theoretical knowledge.")
]

# Variabel kontrol
recording = None
stop_flag = False
skip_countdown_flag = False
current_soal_index = 0
folder_path_rekaman = ""

# =====================================================================
# BAGIAN 2: INISIALISASI EXCEL
# =====================================================================
try:
    workbook = openpyxl.load_workbook(file_path_excel)
    sheet = workbook.active
    if sheet.max_row == 0 or [cell.value for cell in sheet[1]] != header_excel:
        sheet.delete_rows(1, sheet.max_row)
        sheet.append(header_excel)
    workbook.save(file_path_excel)
except FileNotFoundError:
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(header_excel)
    workbook.save(file_path_excel)

# =====================================================================
# BAGIAN 3: WIDGETS FORM & PEREKAM
# =====================================================================
# Form data diri
nama = widgets.Text(description="Nama:", layout=widgets.Layout(width='400px'))
jenis_kelamin = widgets.Dropdown(options=["Laki-laki", "Perempuan"], description="Jenis Kelamin:")
program_studi = widgets.Text(description="Prodi:", layout=widgets.Layout(width='400px'))
asal_kota = widgets.Text(description="Asal Kota:", layout=widgets.Layout(width='400px'))
umur = widgets.IntText(description="Umur:")
domisili = widgets.Text(description="Domisili:", layout=widgets.Layout(width='400px'))
asal_kampus = widgets.Text(description="Asal Kampus:", layout=widgets.Layout(width='400px'))
jenis_test = widgets.Dropdown(options=["TOEFL", "IELTS", "Duolingo", "Lainnya", "Belum Pernah"], description="Jenis Tes:")
skor_test = widgets.IntText(description="Skor Tes:", value=0, disabled=True)
persepsi = widgets.Dropdown(options=["Dasar", "Menengah", "Mahir"], description="Persepsi:")
submit_btn = widgets.Button(description="✅ Lanjut ke Sesi Rekaman", button_style="success")
output_form = widgets.Output()

# Perekam suara & UI HTML
start_btn = widgets.Button(description="▶️ Mulai Rekam Soal 1", button_style="success")
stop_btn = widgets.Button(description="⏹ Stop Rekam", button_style="danger", disabled=True)
skip_btn = widgets.Button(description="⏭️ Lewati Countdown", button_style="info", disabled=True)
countdown_progress = widgets.IntProgress(value=0, min=0, max=default_countdown,
                                         description='Persiapan:',
                                         bar_style='info',
                                         style={'description_width': 'initial'},
                                         layout=widgets.Layout(width='500px', visibility='hidden'))

soal_ui_html = widgets.HTML(value="<div style='padding: 10px; border: 1px solid #e0e0e0; border-radius: 5px; min-height: 100px;'><i>Menunggu sesi rekaman dimulai...</i></div>")

# =====================================================================
# BAGIAN 4: FUNGSI-FUNGSI
# =====================================================================
def handle_test_change(change):
    skor_test.disabled = change['new'] == "Belum Pernah"
    if skor_test.disabled:
        skor_test.value = 0

jenis_test.observe(handle_test_change, names='value')

def record_audio_for_soal(soal_index):
    global recording, stop_flag, skip_countdown_flag, folder_path_rekaman
    soal_name, soal_text = soal_list[soal_index]
    filename = os.path.join(folder_path_rekaman, f"{nama.value.strip().replace(' ', '_')}_{soal_name}.wav")

    def update_ui(status_html, show_countdown=False):
        gambar_html = ""
        if soal_name == "DP":
            gambar_path = "image/describe.jpg"  # ubah sesuai path gambar
            gambar_html = f"<img src='{gambar_path}' style='max-width: 100%; height: auto; border-radius: 5px; margin-bottom: 10px;'>"

        soal_html = f"""
        <div style='border: 1px solid #ccc; padding: 15px; border-radius: 8px; margin-bottom: 10px; background-color: #f9f9f9;'>
            <h4>Soal {soal_index + 1}/{len(soal_list)}: {soal_name}</h4>
            <p style='font-family: serif; font-size: 1.1em; line-height: 1.5;'><i>"{soal_text}"</i></p>
            {gambar_html}
            <hr>
            <p style='text-align: center; font-weight: bold; font-size: 1.2em; margin-top: 15px;'>{status_html}</p>
        </div>
        """
        soal_ui_html.value = soal_html
        countdown_progress.layout.visibility = 'visible' if show_countdown else 'hidden'

    skip_btn.disabled = False

    # Countdown
    for i in range(default_countdown, 0, -1):
        if stop_flag:
            update_ui(f"<span style='color: orange;'>⏹ Rekaman soal '{soal_name}' dibatalkan.</span>")
            skip_btn.disabled = True
            countdown_progress.value = 0
            return
        if skip_countdown_flag:
            update_ui("⏭️ Countdown dilewati!")
            time.sleep(0.5)
            break

        countdown_progress.value = default_countdown - i
        update_ui(f"⏳ Mulai rekam dalam <b>{i} detik...</b>", show_countdown=True)
        time.sleep(1)

    skip_btn.disabled = True
    countdown_progress.value = 0

    if stop_flag: return

    update_ui("<span style='color: red;'>🔴 MEREKAM...</span> Tekan <b>Stop</b> saat selesai.")

    recording_data = []

    def callback(indata, frames, time, status):
        if not stop_flag:
            recording_data.append(indata.copy())

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16', callback=callback):
        while not stop_flag:
            time.sleep(0.1)

    recording = np.concatenate(recording_data, axis=0) if recording_data else np.array([], dtype='int16')

    write(filename, fs, recording)
    update_ui(f"<span style='color: green;'>✅ Rekaman soal '{soal_name}' berhasil disimpan.</span>")
    stop_rekaman(None)

def start_rekaman(b):
    global stop_flag, current_soal_index, skip_countdown_flag
    stop_flag = False
    skip_countdown_flag = False
    start_btn.disabled = True
    stop_btn.disabled = False

    if current_soal_index >= len(soal_list):
        soal_ui_html.value = "<div style='text-align:center; padding: 20px; background-color: #e6ffed; border: 1px solid #b7e4c7; border-radius: 8px;'><h3>🎉 Semua soal sudah direkam. Sesi selesai! ✅</h3></div>"
        start_btn.description = "✅ Selesai"
        stop_btn.disabled = True
        skip_btn.disabled = True
        return

    thread = threading.Thread(target=record_audio_for_soal, args=(current_soal_index,))
    thread.start()

def stop_rekaman(b):
    global stop_flag, current_soal_index
    if not stop_flag:
        stop_flag = True
    stop_btn.disabled = True
    start_btn.disabled = False
    skip_btn.disabled = True

    current_soal_index += 1

    if current_soal_index >= len(soal_list):
        start_btn.description = "✅ Selesai"
        soal_ui_html.value = "<div style='text-align:center; padding: 20px; background-color: #e6ffed; border: 1px solid #b7e4c7; border-radius: 8px;'><h3>🎉 Semua soal sudah direkam. Sesi selesai! ✅</h3></div>"
    else:
        start_btn.description = f"▶️ Lanjut Soal {current_soal_index + 1}"
        # otomatis rekam soal berikutnya
        start_rekaman(None)

def skip_countdown(b):
    global skip_countdown_flag
    skip_countdown_flag = True

def on_submit_form(b):
    global folder_path_rekaman
    if not nama.value.strip():
        with output_form:
            clear_output(wait=True)
            print("⚠️ Nama tidak boleh kosong!")
        return

    sheet.append([
        nama.value, jenis_kelamin.value, program_studi.value, asal_kota.value,
        umur.value, domisili.value, asal_kampus.value, jenis_test.value,
        skor_test.value, persepsi.value
    ])
    workbook.save(file_path_excel)

    folder_path_rekaman = base_folder_rekaman
    os.makedirs(folder_path_rekaman, exist_ok=True)


    clear_output(wait=True)
    display(widgets.HTML(f"<h3>👋 Halo, {nama.value}! Data Anda berhasil disimpan.</h3><p>Folder rekaman Anda: <b>'{folder_path_rekaman}'</b><br>Sekarang, silakan mulai sesi rekaman suara di bawah ini.</p>"))
    display(ui_rekaman)

# =====================================================================
# BAGIAN 5: LAYOUT & EVENT HANDLING
# =====================================================================
submit_btn.on_click(on_submit_form)
start_btn.on_click(start_rekaman)
stop_btn.on_click(stop_rekaman)
skip_btn.on_click(skip_countdown)

form_items = widgets.VBox([
    widgets.HTML("<h3>Formulir Data Peserta</h3>"),
    nama, jenis_kelamin, program_studi, asal_kota, umur, domisili, asal_kampus,
    jenis_test, skor_test, persepsi, submit_btn, output_form
])

ui_rekaman = widgets.VBox([
    widgets.HTML("<hr><h3>Sesi Perekaman Suara</h3>"),
    soal_ui_html,
    countdown_progress,
    widgets.HBox([start_btn, stop_btn, skip_btn]),
])

display(form_items)


In [27]:
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
